In [75]:
import pandas as pd
import numpy as np
import datetime
import time

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# knowing the data

In [13]:
events_df = pd.read_csv('events.csv')
events_df

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [33]:
events_df.keys()

Index(['timestamp', 'visitorid', 'event', 'itemid', 'transactionid'], dtype='object')

In [12]:
events_df['event'].unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

In [34]:
events_df[events_df.transactionid.notnull()].visitorid.unique()    

array([ 599528,  121688,  552148, ..., 1155978, 1050575,  855941])

visitorid of thoes visitors who made transactions 

In [29]:
events_df[events_df.transactionid.notnull()].event.unique()

array(['transaction'], dtype=object)

'transaction' is recorded in event, whenever transactionid(purchase) is created

In [37]:
category_df = pd.read_csv('category_tree.csv')

In [38]:
category_df

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
...,...,...
1664,49,1125.0
1665,1112,630.0
1666,1336,745.0
1667,689,207.0


Categoryid explain the relationship of different products with each other, like categoryid 1016 is a child of parentid 213.

In [40]:
item_properties_1_df = pd.read_csv('item_properties_part1.csv')

In [41]:
item_properties_1_df

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
...,...,...,...,...
10999994,1439694000000,86599,categoryid,618
10999995,1435460400000,153032,1066,n1020.000 424566
10999996,1440298800000,421788,888,35975 856003 37346
10999997,1437879600000,159792,400,n552.000 639502 n720.000 424566


timestamp is still the same Unix format.

itemid is the unique item identifier.

Property is the Item's attributes such as category id and availability while the rest are hashed for confidentiality purposes.

Value is the item's property value like availability(one of the properties) is 1 if there is stock and 0 otherwise.

In [43]:
item_properties_1_df.loc[(item_properties_1_df.property == 'categoryid') & (item_properties_1_df.value == '618')].sort_values('timestamp').head()

,timestamp,itemid,property,value
8582420,1431226800000,123973,categoryid,618
4818247,1431226800000,413487,categoryid,618
2842000,1431226800000,268139,categoryid,618
9943275,1431226800000,146905,categoryid,618
5769012,1431226800000,451095,categoryid,618


above is the number of items under category id 618.

# Customer Behaviour Exploration
Its good to categorise coustomers in (a) how made transactions, (b) thoes who do not made transaxtion.

In [44]:
# all customers who made transaction.
customer_purchased = events_df[events_df.transactionid.notnull()].visitorid.unique()

In [59]:
len(customer_purchased)

11719

these many visitors made purchases.

and their unique 'visitorid' is stored in customer_purchased

In [48]:
all_customers = events_df.visitorid.unique()

In [49]:
len(all_customers)
#all the visitors of the item.

1407580

In [51]:
customer_browsed = [x for x in all_customers if x not in customer_purchased]  # this code takes time

In [62]:
type(customer_browsed),len(customer_browsed)

(list, 1395861)

In [69]:
customer_browsed = np.isin(all_customers,customer_purchased)  

# another way of doing this.
#customer_browsed = np.array(list(set(all_customers)- set(customer_purchased)))  


# both these codes save time, 

In [70]:
len(customer_browsed)

1407580

these many customers have visited the item, but did not purchaced it.

REMEMBER: CUSTOMER_PURCHASED, CUSTOMER_BROWSED, ALL_CUSTOMERS.  ALL CONTAIN UNIQUE 'visitorid', in numpy array

# Below is a snapshot of visitor id 599528 and their buying journey from viewing to transaction (purchase)

In [71]:
events_df[events_df.visitorid == 599528].sort_values('timestamp')

,timestamp,visitorid,event,itemid,transactionid
16729,1433221896102,599528,view,356475,NaN
112,1433221941632,599528,addtocart,356475,NaN
130,1433222276276,599528,transaction,356475,4000.0
29505,1433297545235,599528,view,356475,NaN
114163,1433625010602,599528,view,356475,NaN
145498,1433731497480,599528,view,356475,NaN
187497,1433896449376,599528,view,356475,NaN
171995,1433896477794,599528,view,356475,NaN
197272,1433960259198,599528,view,356475,NaN
192471,1433960567121,599528,view,356475,NaN



Now, that we know about customer_purchased lets find out which items they purchased.


In [109]:
purchased_items =[]
for customer in customer_purchased:
    purchased_items.append(list(events_df.loc[(events_df.visitorid == customer) & (events_df.transactionid.notnull())].itemid.unique()))

purchased_items is a list which contain itemid of purchased items.

In [110]:
len(purchased_items)

11719

In [117]:
purchased_items[100:150]

[[329522],
 [164885],
 [384399],
 [385201],
 [315543],
 [1684],
 [19883],
 [288418],
 [379092],
 [243470],
 [189444],
 [190260],
 [78822, 251415],
 [115530],
 [102573],
 [821],
 [431447],
 [317435],
 [181202],
 [231565],
 [217218, 8641, 32227, 331159],
 [339808],
 [312728,
  353436,
  156552,
  179290,
  182157,
  31058,
  389200,
  152262,
  98425,
  331644,
  454183],
 [448494],
 [199348],
 [118024, 305742, 113641, 275344],
 [281480, 271268, 102408, 358731, 445813, 283037],
 [43485],
 [189764],
 [243980],
 [248614],
 [120098, 34269, 85771, 451501, 7574, 433543, 79801, 291964],
 [127722, 62800, 8641, 185211, 273747, 406121, 403477, 73377, 38437, 340327],
 [52539, 206218, 156767],
 [306328, 249161, 194643],
 [157438, 193554, 428040],
 [8724],
 [219821, 329443],
 [6364,
  331159,
  150889,
  273555,
  406071,
  160272,
  276282,
  100188,
  77844,
  255908,
  388800,
  259710,
  99003,
  252400,
  58123],
 [316753],
 [7943],
 [336359],
 [151471],
 [383396],
 [158575],
 [114121,
  382649

above data is given in the form of list of lists.

The inner lists are collection of itemid purchased by single costumer(possibly on different dates).

This inner list of items can be used to suggest visitors("thoes who buy this also buy following"). Provided the visitor buy any one item from this inner list.

In [112]:
# As purchase is made 'itemid' is pass through this function along with purchased_items(which is calculated above)
def recommender_bought_bought(item_id, purchased_items):
    recommender_list = []
    for x in purchased_items:        # x(inner_list) is a purchased item
        if item_id in x:             # purchased item is in x(inner_list)
            recommender_list += x
    recommender_list = list(set(recommender_list) - set([item_id]))
    
    return recommender_list

In [113]:
# Check: recommender_bought_bought()
recommender_bought_bought(302422, purchased_items)

[105792, 200793, 12836, 80582, 380775, 15335, 400969, 25353, 237753, 317178]

So now we can present to the visitor a list of the other items a customer previously bought along with what item the current visitor is viewing e.g. item number 302422